# import pacakges

In [5]:
base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/enhancer/'
library(data.table)
library(dplyr)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(EnsDb.Hsapiens.v75)
library(clusterProfiler)
library(AnnotationDbi)
library(org.Hs.eg.db)
library(ggplot2)
library(gridExtra)
library(IRdisplay)
library(ChIPpeakAnno)
library(StructuralVariantAnnotation)
library(SingleCellExperiment)
library(anndata)
library(scDblFinder)

txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

setwd(base_dir)

options(width = 188)

In [91]:
data_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch3/'


In [27]:
path = paste0(Sys.getenv('PATH'), ':', '/home/whu78/conda3/envs/feb24/bin:/home/whu78/conda3/envs/feb24/condabin:/hpcf/user_commands:/usr/NX/bin:/cm/local/apps/environment-modules/4.5.3/bin:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/etc:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/bin:/usr/lpp/mmfs/bin:/usr/lpp/mmfs/lib:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/usr/sbin:/cm/local/apps/environment-modules/4.5.3/bin:/usr/sbin:/sbin:/bin:/usr/bin:/etc:/usr/ucb:/usr/local/bin:/usr/local/local_dfs/bin:/usr/bin/X11:/usr/local/sas:/home/whu78/.local/bin:/home/whu78/bin')
path = unique(unlist(strsplit(path, ':')))
path = c(path, '/home/whu78/edirect/')
path = paste(path, collapse = ':')
Sys.setenv('PATH' = path)


In [23]:
path

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/conda3/envs/mar24/bin:/condabin:/home/whu78/conda3/bin:/home/whu78/conda3/envs/feb24/bin:/home/whu78/conda3/envs/feb24/condabin:/hpcf/user_commands:/usr/NX/bin:/cm/local/apps/environment-modules/4.5.3/bin:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/etc:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/bin:/usr/lpp/mmfs/bin:/usr/lpp/mmfs/lib:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/bin:/etc:/usr/ucb:/usr/local/local_dfs/bin:/usr/bin/X11:/usr/local/sas:/home/whu78/.local/bin:/home/whu78/bin"

# import atac-seq design table
samples has been processed here: nb_TE_HSC_ATAC

In [20]:
dsn_atac = fread('../../collab_Farhan/data/dsn_te_atac_merged_bak.txt')
head(dsn_atac[, .(sample_name_manual, srr_id, gsm_id)])

sample_name_manual,srr_id,gsm_id
<chr>,<chr>,<chr>
D4983_HSC_1,SRR2920466,GSM1937376
D4983_HSC_1,SRR2920467,GSM1937377
D4983_HSC_1,SRR2920468,GSM1937378
D4983_HSC_1,SRR2920469,GSM1937379
D4983_HSC_1,SRR2920470,GSM1937380
D4983_HSC_1,SRR2920471,GSM1937381


# import GSE231486 ChIPSeq sample information
ChIP-Seq and HI-C seq, no atac-seq


In [39]:
dsn_gse231486 = fread('/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/enhancer/gse231486_table') 
dsn_gse231486[, assay := sub("(.*)_(.*)_(.*)_(.*)", "\\1", tag)]
dsn_gse231486[, cell_type := sub("(.*)_(.*)_(.*)_(.*)", "\\2", tag)]
dsn_gse231486[, antibody := sub("(.*)_(.*)_(.*)_(.*)", "\\3", tag)]
dsn_gse231486[, rep := sub("(.*)_(.*)_(.*)_(.*)", "\\4", tag)]
dsn_gse231486[, sample_name := paste0(cell_type, '_', antibody, '_', rep)]

dim(dsn_gse231486)


[1] 156   7

In [40]:
dsn_gse231486[, srr_id := {system(paste0('~/edirect/esearch -db sra -query "', gsm_id, '" | efetch -format docsum | accn-at-a-time | grep -i srr | tr [:lower:] [:upper:] | tr "\n" "," '), 
                                   intern = T)}, by = 1:nrow(dsn_gse231486)]
head(dsn_gse231486)
#fwrite(dsn_gse231486, file = 'dsn_gse231486.tsv')


gsm_id,tag,assay,cell_type,antibody,rep,sample_name,srr_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM7277346,ChIPmentation_CMP_ERG_rep1,ChIPmentation,CMP,ERG,rep1,CMP_ERG_rep1,"SRR24388802,"
GSM7277347,ChIPmentation_CMP_ERG_rep2,ChIPmentation,CMP,ERG,rep2,CMP_ERG_rep2,"SRR24388801,"
GSM7277348,ChIPmentation_CMP_ERG_rep3,ChIPmentation,CMP,ERG,rep3,CMP_ERG_rep3,"SRR24388800,"
GSM7277349,ChIPmentation_CMP_FLI1_rep1,ChIPmentation,CMP,FLI1,rep1,CMP_FLI1_rep1,"SRR24388799,"
GSM7277350,ChIPmentation_CMP_FLI1_rep2,ChIPmentation,CMP,FLI1,rep2,CMP_FLI1_rep2,"SRR24388798,"
GSM7277351,ChIPmentation_CMP_FLI1_rep3,ChIPmentation,CMP,FLI1,rep3,CMP_FLI1_rep3,"SRR24388797,"


In [83]:
dsn_gse231486 = fread('dsn_gse231486.tsv')
dsn_gse231486[, srr_ids := sub(',$', '', srr_id)]
#dsn_gse231486 = dsn_gse231486[, .(srr_id = unlist(tstrsplit(sra_id, '_', type.convert = TRUE))), by = 'sample_name']
dsn_gse231486 = dsn_gse231486 %>% 
group_by(sample_name) %>% 
separate_rows(srr_ids, sep = ",") %>% 
filter(!is.na(srr_ids))
dsn_gse231486 = as.data.table(dsn_gse231486)
#dsn_gse231486 = dsn_gse231486[!is.na(srr_id), ]
head(dsn_gse231486)


gsm_id,tag,assay,cell_type,antibody,rep,sample_name,srr_id,srr_ids
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM7277346,ChIPmentation_CMP_ERG_rep1,ChIPmentation,CMP,ERG,rep1,CMP_ERG_rep1,"SRR24388802,",SRR24388802
GSM7277347,ChIPmentation_CMP_ERG_rep2,ChIPmentation,CMP,ERG,rep2,CMP_ERG_rep2,"SRR24388801,",SRR24388801
GSM7277348,ChIPmentation_CMP_ERG_rep3,ChIPmentation,CMP,ERG,rep3,CMP_ERG_rep3,"SRR24388800,",SRR24388800
GSM7277349,ChIPmentation_CMP_FLI1_rep1,ChIPmentation,CMP,FLI1,rep1,CMP_FLI1_rep1,"SRR24388799,",SRR24388799
GSM7277350,ChIPmentation_CMP_FLI1_rep2,ChIPmentation,CMP,FLI1,rep2,CMP_FLI1_rep2,"SRR24388798,",SRR24388798
GSM7277351,ChIPmentation_CMP_FLI1_rep3,ChIPmentation,CMP,FLI1,rep3,CMP_FLI1_rep3,"SRR24388797,",SRR24388797


In [84]:
table(dsn_gse231486$antibody)


    CTCF      ERG     FLI1    GATA2  H3K27ac H3K27me3  H3K4me3      HiC   HIChIP      IgG     LMO2     LYL1     Pol2      PU1    RUNX1    STAG2     TAL1 
      14       14       14       14        4        4        4       16       10        4       14       14       10       14       14       12       14 

# bloodChIP extra
for cell lines chip-seq and atac-seq

In [85]:
dsn = fread('/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/enhancer/table2')
setnames(dsn, c('cell_type', 'antibody', 'pmid', 'accession', 'sra_id'))
dsn[, tag := paste0(cell_type, '_', antibody)]
dsn[, index := 1:nrow(dsn)]
dsn = relocate(dsn, 'index')

cell_lines = c('Kasumi1', 'KG1', 'ME1', 'TSU1621MT')
dsn = dsn[cell_type %in% cell_lines, ]
dsn[, sample_name := paste0(cell_type, '_', antibody, '_rep1')]
#dsn = dsn[, .(srr_id = unlist(tstrsplit(sra_id, '_', type.convert = TRUE))), by = 'sample_name']

#dsn[, cell_type := sub('(.*)_(.*)_(.*)', '\\1', sample_name)]
#dsn[, antibody := sub('(.*)_(.*)_(.*)', '\\2', sample_name)]
#dsn[, rep := sub('(.*)_(.*)_(.*)', '\\3', sample_name)]
dsn = dsn %>% group_by(sample_name) %>% separate_rows(sra_id, sep = '_') %>% filter(!is.na(sra_id)) 
dsn = as.data.table(dsn)
head(dsn)


index,cell_type,antibody,pmid,accession,sra_id,tag,sample_name
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,Kasumi1,ATAC,34010414,E-MTAB-9021,ERR4084850,Kasumi1_ATAC,Kasumi1_ATAC_rep1
1,Kasumi1,ATAC,34010414,E-MTAB-9021,ERR4084851,Kasumi1_ATAC,Kasumi1_ATAC_rep1
1,Kasumi1,ATAC,34010414,E-MTAB-9021,ERR4084852,Kasumi1_ATAC,Kasumi1_ATAC_rep1
2,Kasumi1,ERG,27851970,GSM2026052,SRR3082396,Kasumi1_ERG,Kasumi1_ERG_rep1
3,Kasumi1,FLI1,27851970,GSM2026051,SRR3082395,Kasumi1_FLI1,Kasumi1_FLI1_rep1
4,Kasumi1,GATA2,27851970,GSM2026054,SRR3082398,Kasumi1_GATA2,Kasumi1_GATA2_rep1


In [97]:
dsn = rbind(dsn[, .(sample_name, cell_type, antibody, sra_id)], dsn_gse231486[, .(sample_name, cell_type, antibody, srr_ids)], use.names = F)
setnames(dsn, 'sra_id', 'srr_id')
head(dsn)


sample_name,cell_type,antibody,srr_id
<chr>,<chr>,<chr>,<chr>
Kasumi1_ATAC_rep1,Kasumi1,ATAC,ERR4084850
Kasumi1_ATAC_rep1,Kasumi1,ATAC,ERR4084851
Kasumi1_ATAC_rep1,Kasumi1,ATAC,ERR4084852
Kasumi1_ERG_rep1,Kasumi1,ERG,SRR3082396
Kasumi1_FLI1_rep1,Kasumi1,FLI1,SRR3082395
Kasumi1_GATA2_rep1,Kasumi1,GATA2,SRR3082398


In [87]:
fwrite(dsn, file = 'dsn_step2.tsv')

In [88]:
table(dsn$antibody)


    ATAC     CTCF      ERG     FLI1    GATA2  H3K27ac  H3K27Ac H3K27me3  H3K4me3      HiC   HIChIP      IgG     LMO2     LYL1     Pol2     PU.1      PU1   RNAseq    RUNX1    STAG2 
       7       14       18       18       18        4        3        4        4       16       10        6       18       18       10        4       14        2       18       12 
    TAL1 
      18 

In [152]:
dsn = fread('dsn_step2.tsv')
dsn[, srr_id := sra_id]
head(dsn)


sample_name,cell_type,antibody,sra_id,srr_id
<chr>,<chr>,<chr>,<chr>,<chr>
Kasumi1_ATAC_rep1,Kasumi1,ATAC,ERR4084850,ERR4084850
Kasumi1_ATAC_rep1,Kasumi1,ATAC,ERR4084851,ERR4084851
Kasumi1_ATAC_rep1,Kasumi1,ATAC,ERR4084852,ERR4084852
Kasumi1_ERG_rep1,Kasumi1,ERG,SRR3082396,SRR3082396
Kasumi1_FLI1_rep1,Kasumi1,FLI1,SRR3082395,SRR3082395
Kasumi1_GATA2_rep1,Kasumi1,GATA2,SRR3082398,SRR3082398


In [154]:
data_dir

[1] "/research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch3/"

# download fastq files

In [161]:
dsn[, sample_dir := paste0(data_dir, sample_name, '/')]
dsn[, sra_file := paste0(sample_dir, srr_id, '/', srr_id, '.sra')]  


In [155]:
dsn[, prefetch_cmd := paste0('prefetch ',     srr_id, ' -O ', sample_dir, ' --max-size 40g')]
dsn[, fastqdump_cmd:= paste0('cd ', sample_dir, ' && fasterq-dump ', srr_id, ' --outdir ', sample_dir)]

# if single end seq
dsn[, dump_r := paste0(sample_dir,  '/', srr_id, '.fastq')]
# if double end seq
dsn[, dump_r1 := paste0(sample_dir, '/', srr_id, '_1.fastq')] 
dsn[, dump_r2 := paste0(sample_dir, '/', srr_id, '_2.fastq')]

dsn[, gz_r_cmd := paste0(' gzip ', dump_r)]
dsn[, gz_r1_cmd := paste0(' gzip ', dump_r1)]
dsn[, gz_r2_cmd := paste0(' gzip ', dump_r2)]


In [103]:
write(dsn[grep('SRR', srr_id), prefetch_cmd], file = 'run_batch3_prefetch.sh')
dsn[grep('SRR', srr_id), prefetch_cmd][1]


[1] "prefetch SRR3082396 -O /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch3/Kasumi1_ERG_rep1/ --max-size 40g"

In [133]:
write(dsn[, fastqdump_cmd], file = 'run_batch3_fastq_dump.sh')


In [156]:
table(file.exists(dsn$dump_r1))


FALSE  TRUE 
   64   172 

In [164]:
#
(file.exists(dsn$dump_r1) | file.exists(dsn$dump_r))


[1] FALSE FALSE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [31]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [61]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
 [91]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[121]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[151]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[181]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[211]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE

In [ ]:
# Single End sequenceing or Double end sequencing
dsn[, sede := 'de'] 
dsn[file.exists(dumpr), sede := 'se']
dsn[1:3, sede := 'se']


In [135]:
write(dsn[grepl('SRR', srr_id) & !file.exists(dsn$sra_file), prefetch_cmd], file = 'run_batch3_prefetch.sh')
write(dsn[grepl('SRR', srr_id), fastqdump_cmd], file = 'run_batch3_fastq.sh')


In [162]:
all(file.exists(dsn$sra_file[4:length(dsn$sra_file)]))

[1] TRUE

In [ ]:
prefetch SRR24401764 -O /research_jude/rgs01_jude/groups/jxugrp/home/common/Data_Public/ATAC-seq/Hematopoietic_lineages_batch3/HiC_HSC_rep2_HiC_HSC_rep2_HiC_HSC_rep2/ --max-size 40g


# cutadapt

In [ ]:
# module load cutadapt/4.4

dsn[, r1_trimmed := paste0(sample_dir, sample_name, '_R1_trimmed.fa.gz')]
dsn[, r2_trimmed := paste0(sample_dir, sample_name, '_R2_trimmed.fa.gz')]
dsn[, cutadapt_jobname := paste0('cuta_', sample_name)]
dsn[, cutadapt_oo := paste0(sample_dir, sample_name, '_', cutadapt_jobname, '_out')]
dsn[, cutadapt_eo := paste0(sample_dir, sample_name, '_', cutadapt_jobname, '_err')]
dsn[, cutadapt_cmd := paste0('bsub -J ', cutadapt_jobname, ' -n 10 -o ', cutadapt_oo, ' -e ', cutadapt_eo)]
dsn[file.exists(RR2),  cutadapt_cmd := paste0(cutadapt_cmd, ' " cutadapt -o ', fastq_r1_trimmed, ' -p ', fastq_r2_trimmed, ' ', RR1, ' ', RR2, '"')]
dsn[!file.exists(RR2), cutadapt_cmd := paste0(cutadapt_cmd, ' " cutadapt -o ', fastq_r1_trimmed, ' ', RR1, '"')]
head(dsn[!file.exists(fastq_r1_trimmed) | file.size(fastq_r1_trimmed) < 500000, cutadapt_cmd], 1)
#write(dsn[count_file == 'No', cutadapt_cmd], file = paste0(base_dir, 'run_cutadapt.sh'))
write(dsn[!file.exists(fastq_r1_trimmed) | file.size(fastq_r1_trimmed) < 500000, cutadapt_cmd], file = paste0(wk_dir, 'run_', PP, '_cutadapt.sh'))


In [143]:
file.exists(dsn$r1_trimmed)

ERROR: Error in file.exists(dsn$r1_trimmed): invalid 'file' argument
